In [2]:
# !pip install gurobipy

# from google.colab import drive
# drive.mount('/content/drive')

from gurobipy import *
from itertools import combinations

In [ ]:
# !git clone https://github.com/Nana2929/OR_Final_Project
!rm -r /content/OR_Final_Project
!git clone https://github.com/Nana2929/OR_Final_Project

## real testcases

In [3]:
import joblib
# prefix = '/content/OR_Final_Project/data/testcases'
prefix = './data/testcases'

In [10]:
# small_n = n_node =10
# # slicing 
# d = d[:n_node+1, :n_node+1]
# b = b[:n_node+1]
# b[0] = -sum(b[1:])
# print('toy example n_node:', n_node)
# print('distance matrix shape:', d.shape)
# print('b list length:', len(b))

toy example n_node: 81
distance matrix shape: (82, 82)
b list length: 82


In [4]:
import os
cases = {}
for casepath in os.listdir(prefix):
    if not casepath.startswith('.'):
        case = joblib.load(f'{prefix}/{casepath}')
        cases[casepath] = case

In [4]:
len(cases) # k: 5, capacity: 15, 20, 25, 30 

40

In [ ]:
res= {}
for name, testcase in cases.items():
    if name == 'testcase_even_k5_Q30':
        n_node = testcase['n_node']
        d = testcase['d']
        b = testcase['b']
        Q = testcase['Q']
        n_vehicle = testcase['n_vehicle']
        ###### reduce problem size #######
        small_n = 20
        # depot node is the extra 1
        d = d[:small_n+1, :small_n+1]
        b = b[:small_n+1]
        b[0] = - sum(b[1:]) # -98.0
        n_node = small_n
        ##############
        X, R = VRP(n_vehicle= n_vehicle, n_node = n_node,d = d, Q= Q, b= b)
        print(f'{name} succeeds.')
        res[name] = (X, R)
    

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5245227 rows, 4411 columns and 550515530 nonzeros
Model fingerprint: 0xbffe9e25
Variable types: 2206 continuous, 2205 integer (2205 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 1e+02]


In [15]:
res= {}
for name, testcase in cases.items():
    
    n_node = testcase['n_node']
    d = testcase['d']
    b = testcase['b']
    Q = testcase['Q']
    n_vehicle = testcase['n_vehicle']
    ###### reduce problem size #######
    small_n = 20
    # depot node is the extra 1
    d = d[:small_n+1, :small_n+1]
    b = b[:small_n+1]
    b[0] = - sum(b[1:]) # -98.0
    n_node = small_n
    ##############
    X, R = VRP(n_vehicle= n_vehicle, n_node = n_node,d = d, Q= Q, b= b)
    print(f'{name} succeeds.')
    res[name] = (X, R)


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1049063 rows, 883 columns and 110103106 nonzeros
Model fingerprint: 0x62b5b5de
Variable types: 442 continuous, 441 integer (441 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+02]
Presolve removed 0 rows and 0 columns (presolve time = 18s) ...
Presolve removed 0 rows and 0 columns (presolve time = 29s) ...
Presolve removed 0 rows and 0 columns (presolve time = 30s) ...
Presolve removed 43 rows and 42 columns (presolve time = 36s) ...
Presolve removed 64 rows and 42 columns (presolve time = 52s) ...
Presolve removed 64 rows and 42 columns (presolve time = 65s) ...
Presolve removed 64 rows and 63 columns (presolve time = 65s) ...
Presolve removed 64 rows and 63 columns (presolve time = 88s) ...
Presolve removed 64 rows

AttributeError: Unable to retrieve attribute 'X'

In [19]:
cases.keys()

dict_keys(['testcase_even_k1_Q15', 'testcase_even_k2_Q25', 'testcase_even_k3_Q25', 'testcase_even_k2_Q15', 'testcase_even_k1_Q25', 'testcase_even_k3_Q15', 'testcase_even_k4_Q25', 'testcase_even_k5_Q25', 'testcase_even_k4_Q15', 'testcase_even_k5_Q15', 'testcase_morn_k1_Q20', 'testcase_even_k5_Q30', 'testcase_even_k4_Q30', 'testcase_morn_k2_Q20', 'testcase_morn_k3_Q20', 'testcase_even_k3_Q30', 'testcase_even_k2_Q30', 'testcase_morn_k4_Q20', 'testcase_even_k1_Q30', 'testcase_morn_k5_Q20', 'testcase_morn_k4_Q30', 'testcase_even_k1_Q20', 'testcase_morn_k5_Q30', 'testcase_even_k3_Q20', 'testcase_even_k2_Q20', 'testcase_morn_k2_Q30', 'testcase_morn_k3_Q30', 'testcase_morn_k1_Q30', 'testcase_even_k5_Q20', 'testcase_even_k4_Q20', 'testcase_morn_k3_Q25', 'testcase_morn_k1_Q15', 'testcase_morn_k2_Q25', 'testcase_morn_k3_Q15', 'testcase_morn_k2_Q15', 'testcase_morn_k1_Q25', 'testcase_morn_k5_Q25', 'testcase_morn_k4_Q25', 'testcase_morn_k5_Q15', 'testcase_morn_k4_Q15'])

In [16]:
b[0]

-98.0

In [7]:
def VRP(n_node = n_node, n_vehicle = n_vehicle, d = d, Q = Q, b = b, verbose = True):
    """
    參數定義：
    n_node - 整數，租借站個數（不包含depot node）
    n_vehicle - 整數，卡車數量
    d　- 二維list，大小為n_node + 1 * n_node + 1，d[i][j]為node i到node j的距離，node 0到所有node距離為0
    b - 一維list，大小為n_node + 1, b[j]為node j的淨供給，b[0] = -(所有node總和)
    Q - 整數，一輛卡車能承載的腳踏車數
    """
    vrp = Model("vrp")

    # add variables as a list
    x = []
    r = []
    for k in range(n_vehicle):
        x.append([])
        r.append([])
        for i in range(n_node + 1):
            x[k].append([])
            r[k].append([])
            for j in range(n_node + 1):
                x[k][i].append(vrp.addVar(lb=0, vtype = GRB.BINARY, name='_'.join(["x", str(i), str(j), str(k + 1)])))
                r[k][i].append(vrp.addVar(lb=0, ub=Q, vtype = GRB.CONTINUOUS, name='_'.join(["x", str(i), str(j), str(k + 1)])))
    
    w = vrp.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

    # set obj and Constraint
    vrp.setObjective(w, GRB.MINIMIZE)

    for k in range(n_vehicle):
        for i in range(n_node + 1):
            for j in range(n_node + 1):
                if i == j:
                    vrp.addConstr(x[k][i][j] == 0, '{} != {} Constraint for vehicle{}'.format(i, j, k + 1))
    
    # 進=出
    vrp.addConstrs((quicksum(x[k][i][j] for i in range(n_node + 1)) == quicksum(x[k][j][i] for i in range(n_node + 1)) for j in range(n_node + 1) for k in range(n_vehicle)), 'in=out')
    
    # 起點&終點為0
    vrp.addConstrs((quicksum(x[k][0][j] for j in range(1, n_node + 1)) == 1 for k in range(n_vehicle)), 'start')
    vrp.addConstrs((quicksum(x[k][j][0] for j in range(1, n_node + 1)) == 1 for k in range(n_vehicle)), 'end')

    # 連結r和x，Q為r的upperbound
    vrp.addConstrs((r[k][i][j] <= Q * x[k][i][j] for i in range(n_node + 1) for j in range(n_node + 1) for k in range(n_vehicle)), 'connect r and x')

    # demand fulfillment
    vrp.addConstrs((quicksum(r[k][i][j] for i in range(n_node + 1) for k in range(n_vehicle)) + b[j] == quicksum(r[k][j][i] for i in range(n_node + 1) for k in range(n_vehicle)) for j in range(n_node + 1)), 'demand fulfillment')

    # 載0台出門
    vrp.addConstrs((quicksum(r[k][0][j] for j in range(n_node + 1)) == 0 for k in range(n_vehicle)), 'empty at start')

    # 吸收多餘supply
    vrp.addConstr(quicksum(r[k][j][0] for j in range(n_node + 1) for k in range(n_vehicle)) == quicksum(b[j] for j in range(1, n_node + 1)), 'remaining supply')

    # subtour
    for size in range(2, n_node):  # size of subset
        combs = list(combinations(range(1, n_node + 1), size))  #　subsets
        for c in range(len(combs)):
            vrp.addConstrs((quicksum(x[k][i][j] for i in combs[c] for j in combs[c]) <= len(combs[c]) - 1 for k in range(n_vehicle)), 'subtour')

    # makespan
    vrp.addConstrs((w >= quicksum(d[i][j] * x[k][i][j] for i in range(1, n_node + 1) for j in range(1, n_node + 1)) for k in range(n_vehicle)), 'makespan')
    if not verbose: vrp.setParam('OutputFlag', 0)
    vrp.optimize()

    # retriving sol
    x_ans = []
    r_ans = []
    for k in range(n_vehicle):
        x_ans.append([])
        r_ans.append([])
        for i in range(n_node + 1):
            x_ans[k].append([])
            r_ans[k].append([])
            for j in range(n_node + 1):
                x_ans[k][i].append(x[k][i][j].X)
                r_ans[k][i].append(r[k][i][j].X)
    return x_ans, r_ans

In [4]:
# small test
n_node = 3
n_vehicle = 1
d = [[0, 0, 0, 0], [0, 2, 2, 2], [0, 2, 2, 2], [0, 2, 2, 2]]
Q = 100
b = [-3, 10, -3, -4]

x_test, r_test = VRP(n_node, n_vehicle, d, Q, b)

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 36 rows, 33 columns and 120 nonzeros
Model fingerprint: 0x79dea985
Variable types: 17 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 6.0000000
Presolve removed 20 rows and 18 columns
Presolve time: 0.01s
Presolved: 16 rows, 15 columns, 50 nonzeros
Variable types: 6 continuous, 9 integer (9 binary)

Root relaxation: objective 4.000000e+00, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.0000000    4.000

Restricted license - for non-production use only - expires 2023-10-25


In [ ]:
x_test

[[[0.0, 1.0, -0.0, -0.0],
  [0.0, 0.0, 1.0, -0.0],
  [-0.0, -0.0, 0.0, 1.0],
  [1.0, -0.0, 0.0, 0.0]]]

In [ ]:
r_test

[[[0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 10.0, 0.0],
  [0.0, 0.0, 0.0, 7.0],
  [3.0, 0.0, 0.0, 0.0]]]